# AutoGluon Tabular - Quick Start

![AutoGluon](https://user-images.githubusercontent.com/16392542/77208906-224aa500-6aba-11ea-96bd-e81806074030.png)


+ **`AutoGluon` automatise les tâches d'apprentissage automatique vous permettant d'obtenir facilement de solides performances prédictives dans vos applications. Avec seulement quelques lignes de code, vous pouvez entraîner et déployer des modèles de machine learning et de deep learning de haute précision sur des images, du texte, des séries chronologiques et des données tabulaires.**



In [ ]:
!python -m pip install --upgrade pip
!python -m pip install autogluon

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

## Example Data

+ **Pour ce didacticiel, nous utiliserons un ensemble de données de l'article de couverture de [Nature numéro 7887](https://www.nature.com/nature/volumes/600/issues/7887) : [Intuition guidée par l'IA pour les théorèmes mathématiques]( https://www.nature.com/articles/s41586-021-04086-x.pdf). Le but est de prédire la signature d'un nœud en fonction de ses propriétés. Nous avons échantillonné 10 000 exemples de formation et 5 000 tests à partir des [données originales](https://github.com/deepmind/mathematics_conjectures/blob/main/knot_theory.ipynb). L'ensemble de données échantillonné permet à ce didacticiel de s'exécuter rapidement, mais AutoGluon peut gérer l'ensemble de données complet si vous le souhaitez.**

+ **Nous chargeons cet ensemble de données directement à partir d'une URL. `TabularDataset` d'AutoGluon est une sous-classe de pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), donc toutes les méthodes `DataFrame` peuvent également être utilisées sur `TabularDataset`.**

In [3]:
data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'
train_data = TabularDataset(f'{data_url}train.csv')
train_data.head()

,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


+ **Nos cibles sont stockées dans la colonne "signature", qui comporte 18 entiers uniques. Même si les pandas n'ont pas correctement reconnu ce type de données comme catégorique, AutoGluon résoudra ce problème.**

In [4]:
label = 'signature'
train_data[label].describe()

count    10000.000000
mean        -0.022000
std          3.025166
min        -12.000000
25%         -2.000000
50%          0.000000
75%          2.000000
max         12.000000
Name: signature, dtype: float64

## Training

+ **Nous construisons maintenant un `TabularPredictor` en spécifiant le nom de la colonne d'étiquette, puis nous entraînons sur l'ensemble de données avec `TabularPredictor.fit()`. Nous n'avons pas besoin de spécifier d'autres paramètres. `AutoGluon` reconnaîtra qu'il s'agit d'une tâche de classification multi-classe, effectuera une ingénierie automatique des fonctionnalités, entraînera plusieurs modèles, puis regroupera les modèles pour créer le prédicteur final.**

In [5]:
predictor = TabularPredictor(label=label).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20240212_225757"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240212_225757"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
C

+ L'ajustement du modèle devrait prendre quelques minutes ou moins en fonction de votre processeur. Vous pouvez accélérer la formation en spécifiant l'argument `time_limit`. Par exemple, `fit(..., time_limit=60)` arrêtera l'entraînement après 60 secondes. Des délais plus élevés entraîneront généralement de meilleures performances de prédiction, et des délais excessivement bas empêcheront AutoGluon de former et d'assembler un ensemble raisonnable de modèles.


## Prediction

+ Une fois que nous disposons d'un prédicteur adapté à l'ensemble de données d'entraînement, nous pouvons charger un ensemble de données distinct à utiliser pour la prédiction et l'évaluation.

In [6]:
test_data = TabularDataset(f'{data_url}test.csv')

y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

Loaded data from: https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/test.csv | Columns = 19 / 19 | Rows = 5000 -> 5000


0   -4
1   -2
2    0
3    4
4    2
Name: signature, dtype: int64

## Evaluation

+ Nous pouvons évaluer le prédicteur sur l'ensemble de données de test à l'aide de la fonction `evaluate()`, qui mesure les performances de notre prédicteur sur des données qui n'ont pas été utilisées pour ajuster les modèles.

In [7]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9512,
 'balanced_accuracy': 0.7641435066768523,
 'mcc': 0.9402072998215631}

+ `TabularPredictor` d'AutoGluon fournit également la fonction `leaderboard()`, qui nous permet d'évaluer les performances de chaque modèle formé individuellement sur les données de test.

In [8]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.9512,0.965966,accuracy,2.092208,0.673810,40.506900,0.011815,0.003881,1.772188,2,True,14
1,LightGBM,0.9456,0.955956,accuracy,0.724144,0.143718,10.519800,0.724144,0.143718,10.519800,1,True,5
2,XGBoost,0.9448,0.956957,accuracy,1.987027,0.625069,21.698781,1.987027,0.625069,21.698781,1,True,11
3,LightGBMLarge,0.9444,0.949950,accuracy,2.455817,0.422503,16.094423,2.455817,0.422503,16.094423,1,True,13
4,CatBoost,0.9432,0.955956,accuracy,0.097842,0.020244,76.199011,0.097842,0.020244,76.199011,1,True,8
5,RandomForestEntr,0.9384,0.949950,accuracy,0.391313,0.160500,8.657200,0.391313,0.160500,8.657200,1,True,7
6,ExtraTreesGini,0.9360,0.946947,accuracy,0.588176,0.139087,2.698446,0.588176,0.139087,2.698446,1,True,9
7,ExtraTreesEntr,0.9358,0.942943,accuracy,0.732020,0.139047,2.874857,0.732020,0.139047,2.874857,1,True,10
8,NeuralNetFastAI,0.9354,0.944945,accuracy,0.093367,0.044859,17.035931,0.093367,0.044859,17.035931,1,True,3
9,RandomForestGini,0.9352,0.944945,accuracy,0.376110,0.136958,7.654361,0.376110,0.136958,7.654361,1,True,6


## Conclusion

+ Dans ce didacticiel de démarrage rapide, nous avons vu les fonctionnalités de base d'ajustement et de prédiction d'AutoGluon à l'aide de `TabularDataset` et `TabularPredictor`. AutoGluon simplifie le processus de formation du modèle en ne nécessitant pas d'ingénierie de fonctionnalités ni de réglage des hyperparamètres du modèle. Consultez les didacticiels détaillés pour en savoir plus sur les autres fonctionnalités d'AutoGluon, telles que la personnalisation des étapes de formation et de prédiction ou l'extension d'AutoGluon avec des générateurs de fonctionnalités, des modèles ou des métriques personnalisés.